In [1]:
import urllib.request
import os
import zipfile
from scipy.io import arff
import pandas as pd
import numpy as np
from sktime.classifiers import TSDummyClassifier


In [2]:
TRAIN_SUFFIX = '_TRAIN.arff'
TEST_SUFFIC  = '_TEST.arff'

In [3]:
#download
dtsName = 'GunPoint'
dtsExt = '.zip'
TSCurl = 'http://www.timeseriesclassification.com/Downloads/'
url = f'{TSCurl}{dtsName}{dtsExt}'

proxy_url = 'ldn2tra3:3128'
cache_path = f'.{os.sep}.sktime_temp_data{os.sep}'
store_loc = f'{cache_path}{dtsName}{os.sep}'
file_loc = f'{store_loc}{dtsName}{dtsExt}'
if not os.path.exists(store_loc):
    os.makedirs(store_loc)


if proxy_url is not None:    
    proxy = urllib.request.ProxyHandler({'http': proxy_url})
    opener = urllib.request.build_opener(proxy)
    urllib.request.install_opener(opener)

if not os.path.exists(file_loc):
    urllib.request.urlretrieve(url, file_loc)  

In [4]:
#unzip
zip_ref = zipfile.ZipFile(file_loc, 'r')
zip_ref.extractall(store_loc)
zip_ref.close()

In [5]:
#import from arff
TARGET_ATTRIBUTE_NAME = 'target'
data, meta = arff.loadarff(f'{store_loc}{dtsName}{TRAIN_SUFFIX}')
df = pd.DataFrame(data)
target_ser = pd.Series(df[TARGET_ATTRIBUTE_NAME])
target_ser = target_ser.apply(int)
data_df = df.drop(TARGET_ATTRIBUTE_NAME,axis=1)

In [6]:
num_items = data_df.shape[0]
# sktime_df = pd.DataFrame({'dim1':np.empty(num_items),'target':target_ser})
sktime_df = pd.DataFrame(columns=['dim1','target'])
for i in range(num_items):
        sktime_df.at[i,'dim1'] = 1
        sktime_df.at[i,'dim1'] = data_df.iloc[i,:]
        sktime_df.at[i,'target'] = target_ser.iloc[i]

In [7]:

X = sktime_df['dim1']
y = sktime_df['target']
model = TSDummyClassifier(strategy='most')
model.fit(X, y)
preds = model.predict(X)
